# Handwritten Prescription Medicine Recognition

This notebook implements a TensorFlow-based pipeline for recognizing medicine names from handwritten prescriptions.


In [1]:
# Cell: Imports and Environment Setup
import os
import json
import pathlib
import string
from typing import List, Tuple, Dict

import numpy as np
import pandas as pd
from PIL import Image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)


2025-10-21 05:31:41.994547: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.20.0


In [2]:
# Cell: GPU Configuration
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f"Physical GPUs: {len(gpus)}, Logical GPUs: {len(logical_gpus)}")
    except RuntimeError as e:
        print(e)
else:
    print('No GPU detected. Training will fall back to CPU.')


Physical GPUs: 1, Logical GPUs: 1


I0000 00:00:1761024706.581381    9833 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [3]:
# Cell: Configuration Parameters
DATA_ROOT = pathlib.Path('dataset')  # Root directory containing Training/Validation/Testing
TRAIN_DIR = DATA_ROOT / 'Training'
VAL_DIR = DATA_ROOT / 'Validation'
TEST_DIR = DATA_ROOT / 'Testing'

TRAIN_IMAGES_DIR = TRAIN_DIR / 'training_words'
VAL_IMAGES_DIR = VAL_DIR / 'validation_words'
TEST_IMAGES_DIR = TEST_DIR / 'testing_words'

TRAIN_LABELS_FILE = TRAIN_DIR / 'training_labels.csv'
VAL_LABELS_FILE = VAL_DIR / 'validation_labels.csv'
TEST_LABELS_FILE = TEST_DIR / 'testing_labels.csv'

IMAGE_COLUMN = 'IMAGE'  # column containing the image filename
LABEL_COLUMN = 'MEDICINE_NAME'  # supervised target column; adjust if you want GENERIC_NAME instead

OUTPUT_DIR = pathlib.Path('artifacts')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

IMG_HEIGHT = 128
IMG_WIDTH = 512
BATCH_SIZE = 16
AUTOTUNE = tf.data.AUTOTUNE
MAX_LABEL_LENGTH = 64

print(f'Data root: {DATA_ROOT.resolve()}')
print(f'Training labels: {TRAIN_LABELS_FILE}')
print(f'Validation labels: {VAL_LABELS_FILE}')
print(f'Testing labels: {TEST_LABELS_FILE}')


Data root: /home/mukesh_jat/test/major-project/dataset
Training labels: dataset/Training/training_labels.csv
Validation labels: dataset/Validation/validation_labels.csv
Testing labels: dataset/Testing/testing_labels.csv


In [4]:
# Cell: Placeholder Asset Creation
placeholder_path = pathlib.Path('placeholder.png')
if not placeholder_path.exists():
    placeholder_image = Image.new('L', (IMG_WIDTH, IMG_HEIGHT), color=255)
    placeholder_image.save(placeholder_path)
    print(f'Created placeholder image at {placeholder_path.resolve()}')
else:
    print(f'Placeholder image already exists at {placeholder_path.resolve()}')


Placeholder image already exists at /home/mukesh_jat/test/major-project/placeholder.png


In [ ]:
# Cell: Character Vocabulary
# Build vocabulary from medical lexicon or dataset labels
DEFAULT_CHARSET = string.ascii_lowercase + string.ascii_uppercase + string.digits + ' -./()'

def build_vocabulary(labels: List[str], extra_tokens: str = '') -> Tuple[Dict[str, int], Dict[int, str], int, int]:
    """
    Returns:
      char_to_num: maps each visible character to [0..N-1]
      num_to_char: inverse map
      BLANK_INDEX: index reserved for CTC blank (= N)
      VOCAB_SIZE: N + 1 (includes the CTC blank at the end)
    """
    characters = sorted(set(''.join(labels)) | set(DEFAULT_CHARSET) | set(extra_tokens))
    char_to_num = {ch: i for i, ch in enumerate(characters)}      # 0..N-1
    num_to_char = {i: ch for ch, i in char_to_num.items()}
    BLANK_INDEX = len(characters)                                  # last index is blank
    VOCAB_SIZE = len(characters) + 1
    return char_to_num, num_to_char, BLANK_INDEX, VOCAB_SIZE

def load_labels(annotation_file: pathlib.Path) -> List[str]:
    if not annotation_file.exists():
        print(f"{annotation_file} not found; returning placeholder labels.")
        return ['Paracetamol', 'Ibuprofen']
    df = pd.read_csv(annotation_file)
    if LABEL_COLUMN not in df.columns:
        raise ValueError(f"Expected column '{LABEL_COLUMN}' in {annotation_file}, found {list(df.columns)}")
    return df[LABEL_COLUMN].astype(str).tolist()

raw_labels = load_labels(TRAIN_LABELS_FILE)
CHAR_TO_NUM, NUM_TO_CHAR, BLANK_INDEX, VOCAB_SIZE = build_vocabulary(raw_labels)
print(f'Vocabulary size (incl. CTC blank): {VOCAB_SIZE} | CTC blank index: {BLANK_INDEX}')


Vocabulary size: 69


In [ ]:
# Cell: Data Loading Utilities
def read_image(path: tf.Tensor) -> tf.Tensor:
    image = tf.io.read_file(path)
    image = tf.io.decode_png(image, channels=1)
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image

@tf.function
def preprocess_image(image: tf.Tensor) -> tf.Tensor:
    # Ensure float dtype in [0,1]
    image = tf.image.convert_image_dtype(image, tf.float32)

    # 🔄 Rotate portrait images to landscape if needed
    shape = tf.shape(image)
    height = tf.cast(shape[0], tf.int32)
    width = tf.cast(shape[1], tf.int32)
    image = tf.cond(height > width,
                    lambda: tf.image.rot90(image, k=1),
                    lambda: image)

    # ✅ Force a fixed square size for all images
    image = tf.image.resize(image, [512, 512])
    image.set_shape([512, 512, 1])

    # ✨ Optional normalization / enhancements
    image = tf.clip_by_value(image, 0.0, 1.0)
    image = tf.image.adjust_brightness(image, 0.05)
    image = tf.image.adjust_contrast(image, 1.5)
    return image

@tf.function
def augment_image(image: tf.Tensor) -> tf.Tensor:
    image = tf.image.random_brightness(image, 0.15)
    image = tf.image.random_contrast(image, 0.75, 1.25)
    image = tf.image.rot90(image, k=tf.random.uniform([], minval=0, maxval=4, dtype=tf.int32))
    return image

def encode_label_py(label: str) -> np.ndarray:
    # Map visible characters to [0..N-1]; CTC blank is not used in labels
    encoded = [CHAR_TO_NUM.get(ch, CHAR_TO_NUM.get(' ', 0)) for ch in label.strip()]
    # Truncate to MAX_LABEL_LENGTH
    encoded = encoded[:MAX_LABEL_LENGTH]
    # Pad the rest with -1 (required by keras.backend.ctc_batch_cost)
    pad_len = MAX_LABEL_LENGTH - len(encoded)
    if pad_len > 0:
        encoded = encoded + [-1] * pad_len
    return np.array(encoded, dtype=np.int32)


@tf.function
def prepare_example(path: tf.Tensor, label: tf.Tensor, training: bool = False):
    image = read_image(path)
    image = preprocess_image(image)
    if training:
        image = augment_image(image)
    label_encoded = tf.numpy_function(func=lambda l: encode_label_py(l.decode('utf-8')), inp=[label], Tout=tf.int32)
    label_encoded.set_shape([MAX_LABEL_LENGTH])
    return image, label_encoded


In [7]:
# Cell: Dataset Pipeline
def create_dataset(annotation_file: pathlib.Path, images_dir: pathlib.Path, training: bool = False) -> tf.data.Dataset:
    if not annotation_file.exists() or not images_dir.exists():
        print(f"Either {annotation_file} or {images_dir} is missing. Using placeholder samples.")
        dummy_paths = tf.constant([str(pathlib.Path('placeholder.png').resolve())] * len(raw_labels))
        dummy_labels = tf.constant(raw_labels)
        dataset = tf.data.Dataset.from_tensor_slices((dummy_paths, dummy_labels))
    else:
        df = pd.read_csv(annotation_file)
        if IMAGE_COLUMN not in df.columns:
            raise ValueError(f"Expected column '{IMAGE_COLUMN}' in {annotation_file}, found {list(df.columns)}")
        if LABEL_COLUMN not in df.columns:
            raise ValueError(f"Expected column '{LABEL_COLUMN}' in {annotation_file}, found {list(df.columns)}")
        paths = df[IMAGE_COLUMN].apply(lambda p: str((images_dir / p).resolve())).tolist()
        labels = df[LABEL_COLUMN].astype(str).tolist()
        dataset = tf.data.Dataset.from_tensor_slices((paths, labels))

    dataset = dataset.map(lambda p, l: prepare_example(p, l, training), num_parallel_calls=AUTOTUNE)
    if training:
        dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return dataset

train_ds = create_dataset(TRAIN_LABELS_FILE, TRAIN_IMAGES_DIR, training=True)
val_ds = create_dataset(VAL_LABELS_FILE, VAL_IMAGES_DIR, training=False)
test_ds = create_dataset(TEST_LABELS_FILE, TEST_IMAGES_DIR, training=False)


In [8]:
# ✅ Fixed CRNN Model Architecture for 512×512 Images
def build_crnn_model(img_width: int, img_height: int, vocab_size: int) -> keras.Model:
    # Input shape updated for 512×512 images
    input_img = layers.Input(shape=(img_height, img_width, 1), name='image_input')

    # --- Convolutional feature extractor ---
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(input_img)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 1))(x)

    x = layers.Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 1))(x)

    x = layers.Conv2D(512, (2, 2), activation='relu')(x)
    x = layers.BatchNormalization()(x)

    # --- Reshape feature map for sequence modeling ---
    x = layers.Permute((2, 1, 3))(x)                # (batch, width, height, channels)
    x = layers.TimeDistributed(layers.Flatten())(x) # (batch, width, height*channels)

    # --- Recurrent layers ---
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Bidirectional(layers.LSTM(256, return_sequences=True))(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Bidirectional(layers.LSTM(256, return_sequences=True))(x)

    # --- Output layer ---
    output = layers.Dense(vocab_size, activation='softmax', name='dense_output')(x)

    model = keras.Model(inputs=input_img, outputs=output, name='crnn_model')
    return model


# ✅ Build and summarize the model
IMG_HEIGHT = 512
IMG_WIDTH = 512

crnn_model = build_crnn_model(IMG_WIDTH, IMG_HEIGHT, VOCAB_SIZE)
crnn_model.summary()

Model: "crnn_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ image_input (InputLayer)        │ (None, 512, 512, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 512, 512, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 256, 256, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 128, 128, 256)  │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128, 128, 256)  │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 64, 128, 256)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 64, 128, 512)   │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64, 128, 512)   │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 32, 128, 512)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 31, 127, 512)   │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 31, 127, 512)   │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ permute (Permute)               │ (None, 127, 31, 512)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 127, 15872)     │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 127, 64)        │     1,015,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 127, 512)       │       657,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 127, 512)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 127, 512)       │     1,574,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_output (Dense)            │ (None, 127, 69)        │        35,397 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,887,621 (22.46 MB)

 Trainable params: 5,885,061 (22.45 MB)

 Non-trainable params: 2,560 (10.00 KB)

In [9]:
# ✅ CTC Loss and Model Wrapper
class CTCLayer(layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred):
        batch_len = tf.cast(tf.shape(y_true)[0], dtype='int64')
        input_length = tf.cast(tf.shape(y_pred)[1], dtype='int64')
        label_length = tf.cast(tf.shape(y_true)[1], dtype='int64')

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype='int64')
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype='int64')

        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(tf.reduce_mean(loss))
        return y_pred


def build_ctc_model(base_model: keras.Model) -> keras.Model:
    labels = layers.Input(name='label', shape=(MAX_LABEL_LENGTH,), dtype='int32')
    y_pred = base_model.output
    output = CTCLayer(name='ctc_loss')(labels, y_pred)
    ctc_model = keras.Model(inputs=[base_model.input, labels], outputs=output)
    return ctc_model


ctc_model = build_ctc_model(crnn_model)
ctc_model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4))


In [10]:
import tensorflow as tf

# Shows which device is being used for ops
with tf.device('/device:GPU:0'):
    a = tf.random.uniform((1000, 1000))
    b = tf.random.uniform((1000, 1000))
    c = tf.matmul(a, b)
print("✅ TensorFlow successfully executed a matmul on:", c.device)


✅ TensorFlow successfully executed a matmul on: /job:localhost/replica:0/task:0/device:GPU:0


In [11]:
for img, lbl in train_ds.take(1):
    lbl_np = lbl.numpy()
    print(lbl_np[0])  # check one label sequence


[27 47 62 57 60 43 54  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


2025-10-21 05:31:51.821867: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [12]:
# Cell: Training Loop
EPOCHS = 50
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        logits = crnn_model(images, training=True)
        ctc_layer = CTCLayer()
        _ = ctc_layer(labels, logits)
        loss = tf.add_n(ctc_layer.losses) if ctc_layer.losses else 0.0
    gradients = tape.gradient(loss, crnn_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, crnn_model.trainable_variables))
    return loss

print('Custom training step defined. Prefer using ctc_model.fit with tf.data pipelines for full training.')
# Example usage with Keras fit:
history = ctc_model.fit(
    train_ds.map(lambda img, lbl: ({'image_input': img, 'label': lbl}, lbl)),
    validation_data=val_ds.map(lambda img, lbl: ({'image_input': img, 'label': lbl}, lbl)),
    epochs=EPOCHS,
)


Custom training step defined. Prefer using ctc_model.fit with tf.data pipelines for full training.
Epoch 1/50


2025-10-21 05:32:04.098867: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91400
2025-10-21 05:32:17.788656: W tensorflow/core/framework/op_kernel.cc:1855] OP_REQUIRES failed at ctc_loss_op.cc:216 : INVALID_ARGUMENT: Saw a non-null label (index >= num_classes - 1) following a null label, batch: 0 num_classes: 69 labels: 19,57,56,43,68,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0 labels seen so far: 19,57,56,43
2025-10-21 05:32:17.789984: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Saw a non-null label (index >= num_classes - 1) following a null label, batch: 0 num_classes: 69 labels: 19,57,56,43,68,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0 labels seen so far: 19,57,56,43
	 [[{{function_node __inference_one_step_on_data_13709}}{{node fu

InvalidArgumentError: Graph execution error:

Detected at node functional_1/ctc_loss_1/CTCLoss defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/mukesh_jat/test/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/mukesh_jat/test/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/mukesh_jat/test/.venv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 758, in start

  File "/home/mukesh_jat/test/.venv/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/usr/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/usr/lib/python3.12/asyncio/base_events.py", line 1987, in _run_once

  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run

  File "/home/mukesh_jat/test/.venv/lib/python3.12/site-packages/ipykerne

In [ ]:
# Cell: Decoding Utilities
@tf.function
def greedy_decode(pred):
    return tf.math.argmax(pred, axis=-1, output_type=tf.int32)


def decode_batch_predictions(pred):
    results = []
    for text in pred:
        text = tf.gather(text, tf.where(tf.not_equal(text, 0)))
        text = tf.squeeze(text, axis=-1)
        chars = [NUM_TO_CHAR.get(int(char), '') for char in text.numpy()]
        results.append(''.join(chars))
    return results


def recognize_medicines(model: keras.Model, dataset: tf.data.Dataset) -> List[List[str]]:
    medicines = []
    for batch_images, _ in dataset:
        preds = model.predict(batch_images)
        decoded = decode_batch_predictions(greedy_decode(preds))
        medicines.append(decoded)
    return medicines

print('Decoding utilities ready.')


In [ ]:
# Cell: Inference Example
def run_inference_example(model: keras.Model, sample_paths: List[str]):
    for path in sample_paths:
        path_obj = pathlib.Path(path)
        if not path_obj.exists():
            print(f'Sample {path} not found. Skipping.')
            continue
        image = tf.io.read_file(str(path_obj))
        image = tf.io.decode_png(image, channels=1)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = preprocess_image(image)
        image = tf.expand_dims(image, axis=0)
        preds = model.predict(image)
        decoded = decode_batch_predictions(greedy_decode(preds))
        print(f'{path}: {decoded[0]}')

print("Call `run_inference_example(crnn_model, ['path/to/image.png'])` after training.")


## Next Steps

1. Replace placeholder dataset paths with actual annotated prescription data.
2. Ensure labels are properly encoded and aligned with the vocabulary.
3. Train the model using `ctc_model.fit`.
4. Integrate a medicine lexicon for post-processing corrections.


## Dataset Paths

The training pipeline expects the dataset to be organized with the following directories relative to the project root:

- Training images: `dataset/Training/training_words`
- Validation images: `dataset/Validation/validation_words`
- Testing images: `dataset/Testing/testing_words`

CSV annotation files with the medicine labels are expected at:

- Training labels: `dataset/Training/training_labels.csv`
- Validation labels: `dataset/Validation/validation_labels.csv`
- Testing labels: `dataset/Testing/testing_labels.csv`


## Train Model

Fine-tune the CRNN directly from this notebook once the dataset folders are populated at:

- `dataset/Training/training_words` with labels in `dataset/Training/training_labels.csv`
- `dataset/Validation/validation_words` with labels in `dataset/Validation/validation_labels.csv`
- `dataset/Testing/testing_words` with labels in `dataset/Testing/testing_labels.csv`

The code below prepares remapped datasets, configures checkpoints and TensorBoard logging under `artifacts/`, and runs `ctc_model.fit`. Adjust the hyperparameters as needed for your hardware and data volume.


### Troubleshooting Training

If `ctc_model.fit` fails or exits early, walk through these checks before rerunning training:

1. **Verify dataset paths** – confirm the `dataset/Training`, `dataset/Validation`, and `dataset/Testing` folders exist and contain the expected `*_words` subdirectories and CSV files. The helper cell below will print counts if everything is wired correctly.
2. **Inspect CSV columns** – the annotation files must include both the `IMAGE` filename column and the `MEDICINE_NAME` label column. Any missing or misspelled header will raise a ValueError when building the datasets.
3. **Spot-check a batch** – TensorFlow will fall back to the placeholder samples if directories are empty or paths are wrong. Review the sanity-check output and confirm you see real image paths, not `placeholder.png`.
4. **Watch system resources** – large images or high batch sizes may exhaust GPU/CPU RAM. Reduce `BATCH_SIZE`, close other GPU jobs, or run on a machine with more memory if you observe OOM errors.
5. **Confirm TensorFlow availability** – ensure the environment has TensorFlow installed with GPU support if desired (e.g., `pip install tensorflow==2.12.*` or `tensorflow-gpu`). Restart the kernel after installation.
6. **Resume from checkpoints** – if training stops mid-way, reload the best checkpoint by calling `crnn_model.load_weights('artifacts/checkpoints/<file>.keras')` before restarting to avoid losing progress.

Once these checks pass, rerun the callback configuration and the training cell.


In [ ]:

# Cell: Dataset Sanity Checks
SUPPORTED_IMAGE_EXTS = ('.png', '.jpg', '.jpeg', '.bmp')

def inspect_split(images_dir, labels_file, split_name):
    print(f'[{split_name}]')
    if not images_dir.exists():
        print(f'  ✗ Images directory missing: {images_dir}')
    else:
        image_files = [p for p in images_dir.iterdir() if p.suffix.lower() in SUPPORTED_IMAGE_EXTS]
        sample_files = sorted([p.name for p in image_files[:3]])
        print(f'  ✓ Images directory found ({len(image_files)} files with supported extensions)')
        if sample_files:
            print('  Sample image files:', ', '.join(sample_files))
        else:
            print('  ⚠️ No files detected with extensions', SUPPORTED_IMAGE_EXTS)
    if not labels_file.exists():
        print(f'  ✗ Labels CSV missing: {labels_file}')
        return
    df = pd.read_csv(labels_file)
    print(f'  ✓ Labels CSV found with {len(df)} rows and columns: {list(df.columns)}')
    missing_cols = [col for col in [IMAGE_COLUMN, LABEL_COLUMN] if col not in df.columns]
    if missing_cols:
        print(f'  ✗ Missing expected columns: {missing_cols}')
    else:
        preview = df[[IMAGE_COLUMN, LABEL_COLUMN]].head(3)
        print('  Preview:')
        for _, row in preview.iterrows():
            print(f"    - {row[IMAGE_COLUMN]} -> {row[LABEL_COLUMN]}")

inspect_split(TRAIN_IMAGES_DIR, TRAIN_LABELS_FILE, 'Training split')
inspect_split(VAL_IMAGES_DIR, VAL_LABELS_FILE, 'Validation split')
inspect_split(TEST_IMAGES_DIR, TEST_LABELS_FILE, 'Test split')



In [ ]:
checkpoint_dir = OUTPUT_DIR / 'checkpoints'
tensorboard_log_dir = OUTPUT_DIR / 'logs'
checkpoint_dir.mkdir(parents=True, exist_ok=True)
tensorboard_log_dir.mkdir(parents=True, exist_ok=True)

train_inputs = train_ds.map(lambda img, lbl: ({'image_input': img, 'label': lbl}, lbl))
val_inputs = val_ds.map(lambda img, lbl: ({'image_input': img, 'label': lbl}, lbl))
test_inputs = test_ds.map(lambda img, lbl: ({'image_input': img, 'label': lbl}, lbl))

print(f'Checkpoints directory: {checkpoint_dir.resolve()}')
print(f'TensorBoard logs directory: {tensorboard_log_dir.resolve()}')


In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=str(checkpoint_dir / 'crnn_{epoch:02d}.keras'),
        monitor='val_loss',
        mode='min',
        save_best_only=True,
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
    ),
    keras.callbacks.TensorBoard(log_dir=str(tensorboard_log_dir)),
]

history = ctc_model.fit(
    train_inputs,
    validation_data=val_inputs,
    epochs=EPOCHS,
    callbacks=callbacks,
)
